# Using `h3-py` to convert between polygons and H3 cells

We can use `h3-py` to convert GeoJSON-like Polygons and MultiPolygons to sets of H3 cells and vice versa.

First we'll import relevant libraries, and define a plotting helper function to visualize the shapes we're dealing with.

In [ ]:
import h3

import geopandas
import geodatasets
import contextily as cx
import matplotlib.pyplot as plt

def plot_df(df, column=None, ax=None):
    'Plot based on the `geometry` column of a GeoPandas dataframe'
    df = df.copy()
    df = df.to_crs(epsg=3857) # web mercator

    if ax is None:
        fig, ax = plt.subplots(figsize=(8,8))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    df.plot(
        ax=ax,
        alpha=0.5, edgecolor='k',
        column=column, categorical=True,
        legend=True, legend_kwds={'loc': 'upper left'}, 
    )
    cx.add_basemap(ax, crs=df.crs, source=cx.providers.CartoDB.Positron)

def plot_shape(shape, ax=None):
    df = geopandas.GeoDataFrame({'geometry': [shape]}, crs='EPSG:4326')
    plot_df(df, ax=ax)

def plot_cells(cells, ax=None):
    shape = h3.cells_to_h3shape(cells)
    plot_shape(shape, ax=ax)

We start with a GeoPandas `GeoDataFrame` describing the five New York city boroughs.

In [ ]:
df = geopandas.read_file(geodatasets.get_path('nybb'))
df

In [ ]:
plot_df(df, column='BoroName')

## Use a compatible CRS before applying H3 functions

The function `h3.geo_to_cells(geo, res)` takes some `geo` object that implements the `__geo_interface__` (https://gist.github.com/sgillies/2217756) — like a Shapely Polygon or MultiPolygon — and converts it to the set of cells whose centroids are contained in `geo`.

**Note**: Be careful about what CRS you are using. `h3-py` expects coordinates as latitude-longitude pairs. The CRS of the current dataframe (EPSG:2263) gives coordinates in feet! You should first convert the data to something compatible. A common choice is EPSG:4326/WGS84. You'll get incorrect results if you apply `h3.geo_to_cells` without converting.

In [ ]:
# Original CRS
df.crs

In [ ]:
# Converting to EPSG:4326/WGS84
df = df.to_crs(epsg=4326)
df.crs

In [ ]:
# Note that the `geometry` column now has coordinates in degrees
df

## Converting a geo to a collection of H3 cells

First, we select one of the boroughs and get the Shapely `MultiPolygon` describing it.

In [ ]:
geo = df.geometry[0]
type(geo)

We can use `h3.geo_to_cells()` to get the associated set of H3 cells at resolution 6.

In [ ]:
cells = h3.geo_to_cells(geo, res=6)
cells

# Visualizing H3 cells

To visualize these cells, we can convert the set back to a Polygon or MultiPolygon using `h3.cells_to_shape()`.
Note that the returned object will be either `H3Poly` or `H3MulitPoly`, each of which implement `__geo_interface__`.
Because they impelement this interface, these objects will be compatible with Python libraries like GeoPandas.

In [ ]:
h3shape = h3.cells_to_h3shape(cells)
h3shape

In [ ]:
h3shape.__geo_interface__

To demonstrate, we'll convert all of the boroughs and plot the results.

In [ ]:
res = 8

s = df.geometry.apply(lambda x: h3.geo_to_cells(x, res))
s

In [ ]:
s = s.apply(h3.cells_to_h3shape)
s

In [ ]:
s[1]

In [ ]:
df.geometry = s
df

In [ ]:
# TODO: note that geopandas automaticallyc converst the H3Shape objects to Shapely objects via `__geo_interface__`
type(df.geometry[1])

In [ ]:
plot_df(df, column='BoroName')

# H3Poly and H3MultiPoly details

As demonstrated above, we expect `h3.geo_to_cells()` and `h3.cells_to_geo()` to cover most common use cases.
For more control, the API provides some additional objects and functions.

The following classes describe GeoJSON-like Polygons and MultiPolygons:

- `H3Shape`: abstract base class; parent of the two below
- `H3Poly`: similar to a Shapely `Polygon`
- `H3MultiPoly`: similar to a Shapely `MultiPolyon`


The following functions convert between `H3Shape` objects and cells:

- `h3shape_to_cells`
- `cells_to_h3shape`


The following functions convert between "geo" objects and `H3Shape` objects.

- `geo_to_h3shape`
- `h3shape_to_geo`

Here, a "geo" object is either:

- anything that implements `__geo_interface__`
- a dictionary like what is output by `__geo_interface__`

Note that if an object `a` is an `H3Shape` (either `H3Poly` or `H3MultiPoly`)
then the following two expressions are equivalent:

- `h3shape_to_geo(a)`
- `a.__geo_interface__`

Also note that if `a` is an `H3Shape`, then `a` "passes through" `geo_to_h3shape`: `geo_to_h3shape(a) == a`.


Summarizing, `H3Shape` objects are the intermediate representation used to translate from
external objects that implement `__geo_interface__` (like you'd have from using GeoPandas).

```
"Geo" object <-> H3Shape <-> H3 cells
```

In [ ]:
h3.H3Poly?

In [ ]:
# TODO
h3.H3MultiPoly?

In [ ]:
# can iterate through multipolygons
# can get len
# how to read the description
# h3 polygons don't close. order doesn't matter. not differences when converting to geojson

# plot shape helper

# H3Poly and H3MultiPoly

We can create a simple `H3Poly` object by providing a list of the **latitude/longitude pairs** that describe its exterior.

In [ ]:
outer = [
    (37.804, -122.412),
    (37.778, -122.507),
    (37.733, -122.501)
]

hole1 = [
    (37.782, -122.449),
    (37.779, -122.465),
    (37.788, -122.454),
]

hole2 = [
    (37.771, -122.484),
    (37.761, -122.481),
    (37.758, -122.494),
    (37.769, -122.496),
]

poly = h3.H3Poly(outer)
print(poly)
plot_shape(poly)

Holes can be added to the polygon by appending additional lat/lng lists.

In [ ]:
poly = h3.H3Poly(outer, hole1)
print(poly)
plot_shape(poly)

In [ ]:
poly = h3.H3Poly(outer, hole1, hole2)
print(poly)
plot_shape(poly)

The `H3Poly` string representation given by its `__repr__` shows the number of vertices in the outer loop of the polygon, followed
by the number of vertices in each hole.

- `H3Poly.outer` gives the list of lat/lng points making up the outer loop of the polygon.
- `H3Poly.holes` gives each of the lists of lat/lng points making up the holes of the polygon.

In [ ]:
poly = h3.H3Poly(outer, hole1, hole2)
poly

In [ ]:
poly.outer

In [ ]:
poly.holes

`H3Poly.__geo_interface__` gives a GeoJSON representation of the polygon as described in https://gist.github.com/sgillies/2217756

**Note the differences in this representation**: Points are given in **lng/lat order**, and the last vertex repeats the first.

In [ ]:
poly.__geo_interface__

Note that we can create an `H3Poly` object from a GeoJSON-like dictionary using `h3.geo_to_h3shape()`.

In [ ]:
h3.geo_to_h3shape(poly.__geo_interface__)

In [ ]:
h3.geo_to_h3shape(poly.__geo_interface__).__geo_interface__

## H3 Polygons don't need to follow the right-hand rule

`H3Poly` objects do not need to follow the "right-hand rule", unlike GeoJSON Polygons. 
The right-hand rule requires that vertices in outer loops are listed in counterclockwise
order and holes are listed in clockwise order.
`h3-py` accepts loops in any order and will do the correct thing when, for example,
converting to sets of cells. However, `h3-py` won't re-order your loops to
conform to the right-hand rule, so be careful if you're using `__geo_interface__` to plot them.

This is only a concern when creating `H3Poly` objects from external input; `H3Poly` or `H3MultiPoly`
objects created through `h3.cells_to_shape()` **will respect the right-hand rule**.

For example, if we reverse the order of one of the holes in our example polygon,
the hole won't be rendered correctly, but the conversion to cells will still be correct.

In [ ]:
def plot_shape_and_cells(shape, res=9):
    fig, axs = plt.subplots(1,2, figsize=(10,5), sharex=True, sharey=True)
    plot_shape(shape, ax=axs[0])
    plot_cells(h3.h3shape_to_cells(shape, res), ax=axs[1])
    fig.tight_layout()

In [ ]:
# Respects right-hand rule
poly = h3.H3Poly(outer, hole1, hole2)
plot_shape_and_cells(poly, res=10)

In [ ]:
# Does not respect right-hand-rule; second hole is reversed
# Conversion to cells still works, tho!
poly = h3.H3Poly(outer, hole1[::-1], hole2)
plot_shape_and_cells(poly, res=10)

In [ ]:
# Does not respect right-hand-rule; outer loop and second hole are both reversed
# Conversion to cells still works, tho!
poly = h3.H3Poly(outer[::-1], hole1[::-1], hole2)
plot_shape_and_cells(poly, res=10)

## H3MultiPoly

An `H3MultiPoly` can be created from `H3Poly` objects. The string representation of the `H3MultiPoly`
gives the number of vertices in the outer loop of each `H3Poly`, along with the number of vertices
in each hole (if there are any).

In [ ]:
poly1 = h3.H3Poly([(37.804, -122.412), (37.778, -122.507), (37.733, -122.501)])
poly2 = h3.H3Poly(
    [(37.803, -122.408), (37.736, -122.491), (37.738, -122.380), (37.787, -122.39)],
    [(37.760, -122.441), (37.772, -122.427), (37.773, -122.404), (37.758, -122.401), (37.745, -122.428)]
)
mpoly = h3.H3MultiPoly(poly1, poly2)

print(poly1)
print(poly2)
print(mpoly)

In [ ]:
plot_shape(mpoly)

`h3.h3shape_to_cells()` works on both `H3MultiPoly` and `H3Poly` objects.

In [ ]:
cells = h3.h3shape_to_cells(mpoly, res=9)
plot_cells(cells)

In [ ]:
len(mpoly)

In [ ]:
for p in mpoly:
    print(p)

In [ ]:
list(mpoly)

# Summary

geo <> shape <> cells
```

Most users can use

- `geo_to_cells`
- `cells_to_geo`


For an alternative interface, consider the classes

- `H3Shape`
- `H3Poly`
- `H3MultiPoly`


We have a few functions for dealing with polygons in H3:

- `h3shape_to_cells`
- `cells_to_h3shape`

do i need:

- `geo_to_h3shape`
- `h3shape_to_geo`

Define `geo`:

- anything that implements `__geo_interface__`
- a dictionary like `__geo_interface__`

# Interfacing with GeoPandas and other libraries

TODO